In [2]:
import praw
import datetime
import nltk
from collections import defaultdict
from nltk.probability import FreqDist
from string import punctuation
from pprint import pprint
import random

In [3]:
# Reddit credentials

reddit = praw.Reddit(client_id='bloop',
                     client_secret='bloop',
                     user_agent='transcipt by u/jmcnea',
                     username='jmcnea',
                     password='bloopbloopbloop')

In [4]:
candidates_subreddits = ['sandersforpresident',
                        'elizabethwarren',
                        'pete_buttigieg',
                        'joebiden',
                        'the_donald']

In [62]:
### Comments


with open("C://Users//Jordan//Documents//Word Documents//AppliedData//reddit_comments.txt", 'w') as df:
    
    df.write('subreddit\tsubmission_id\tcomment_id\tcomment\tup_votes\tdown_votes\tusername\tdate\n')
    
    for candidate in candidates_subreddits:
        
        subreddit = reddit.subreddit(candidate)
        hot_python = subreddit.hot(limit=None)

        for submission in hot_python:
            if not submission.stickied:

                comments = submission.comments.replace_more(limit=None)

                for idx, comment in enumerate(submission.comments.list()):

                    if comment.stickied:
                        continue

                    if comment.body == '[removed]' or comment == '[deleted]':
                        continue
                                           
                    
                    
                    # this is writing fine, but the second line looks funky.
                    df.write(f'{candidate}\t{submission}\t{comment}\t'+
                    f'{comment.body.encode("utf-8",errors="ignore")}\t{comment.ups}\t{comment.downs}\t{comment.author}'+
                    f'\t{str(datetime.datetime.fromtimestamp(comment.created))}\n')

                
                
                

In [5]:
cand_dict = dict()

for cand in candidates_subreddits:
    cand_dict[cand] = [0] * 10
    cand_dict[cand][2] = ''
    cand_dict[cand][3] = dict()
    cand_dict[cand][4] = list()


In [6]:
stopwords = set(nltk.corpus.stopwords.words("english"))
stopwords_sp = set(nltk.corpus.stopwords.words("spanish"))

def clean_list(text) :
    ''' takes a list of text and returns a new list with 
        * words cast to lowercase
        * stopwords removed
        * only alphanumeric words
    '''
    text_clean = [w.lower() for w in text if w.isalpha()]
    text_clean = [w for w in text_clean if w not in stopwords]
    text_clean = [w for w in text_clean if w not in stopwords_sp]
    return(text_clean)

In [7]:
def desc_features(the_description) :
    """ Input: A twitter description
        Output: A dictionary listin the words that are in 
                the description.
                
        This function does some cleaning on the descriptions,
        removing some punctuation, splitting on whitespace, 
        dropping to lower case. It returns a dictionary 
        of the form 
            {example : True,
             word :    True}
    
        """
    
    
    
    
    exclude = set(punctuation)
    exclude.remove("#") #useful for twitter...
    
    # Found this at https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
    the_description = ''.join([ch.lower() for ch in the_description if ch not in exclude])
    
    word_list = the_description.split()

    ret_val = {}
    
    for word in word_list :
        ret_val[word] = True
    
    return(ret_val)

In [8]:
with open("C://Users//Jordan//Documents//Word Documents//AppliedData//reddit_comments.txt", 'rb') as df:
    
    for idx, row in enumerate(df):
        if idx == 0:
            continue
        
       
        row = row.decode("utf-8").split('\t')
        
        row[3] = row[3][1:]
        cand_dict[row[0]][2] += row[3]
        
        
        row[3] = row[3].split(' ')
        cand_dict[row[0]][0] += len(row[3])
        cand_dict[row[0]][1] += 1
        
        
        cand_words = clean_list([w.lower() for w in row[3]])
                
        cand_dict[row[0]][3] = desc_features(" ".join(cand_words))
        
        cand_dict[row[0]][4].append(" ".join(cand_words))

        
              

In [17]:
for cand in cand_dict:
    cand_words = [w.lower() for w in cand_dict[cand][2].split(' ')]
    print(f'r/{cand} summary statistics:')
    print(f'total words used: {len(cand_words)}')
    print('tokens used: ' + str(len(set(cand_words))))
    print(f'comments: {cand_dict[cand][1]}')
    print(f'words per comment: {(cand_dict[cand][0]/cand_dict[cand][1]):.2f}')
    print(f'lexical diversity: {(len(cand_words)/len(set(cand_words))):.2f}')
    print('percent impeachment: '+
          f'{(100 * cand_words.count("impeachment")/len(cand_words)):.5f}')
    print('percent healthcare: '+
          f'{(100 * cand_words.count("healthcare")/len(cand_words)):.5f}')
    print('-'*40)

r/sandersforpresident summary statistics:
total words used: 610871
tokens used: 63930
comments: 17872
words per comment: 35.18
lexical diversity: 9.56
percent impeachment: 0.00311
percent healthcare: 0.07956
----------------------------------------
r/elizabethwarren summary statistics:
total words used: 1049757
tokens used: 81469
comments: 21097
words per comment: 50.76
lexical diversity: 12.89
percent impeachment: 0.00572
percent healthcare: 0.07145
----------------------------------------
r/pete_buttigieg summary statistics:
total words used: 1000537
tokens used: 87765
comments: 25849
words per comment: 39.71
lexical diversity: 11.40
percent impeachment: 0.00770
percent healthcare: 0.03078
----------------------------------------
r/joebiden summary statistics:
total words used: 252581
tokens used: 30465
comments: 6661
words per comment: 38.92
lexical diversity: 8.29
percent impeachment: 0.00871
percent healthcare: 0.05978
----------------------------------------
r/the_donald summary 

In [60]:
for idx, cand in enumerate(cand_dict):
    
    cand_words = clean_list([w.lower() for w in cand_dict[cand][2].split(' ')])
    cand_fd = FreqDist(cand_words)
    
    print(f'r/{cand} most common words: ')
    print(cand_fd.most_common(50))
    print('*'*40) 
       

r/sandersforpresident most common words: 
[('bernie', 8448), ('people', 8334), ('like', 7251), ('would', 7065), ('get', 4959), ('think', 4674), ('even', 3732), ('one', 3714), ('make', 3021), ('know', 2985), ('also', 2901), ('much', 2736), ('really', 2667), ('want', 2640), ('going', 2571), ('good', 2562), ('right', 2553), ('see', 2523), ('us', 2472), ('need', 2415), ('could', 2412), ('still', 2403), ('warren', 2343), ('sanders', 2298), ('way', 2226), ('time', 2037), ('vote', 1980), ('money', 1935), ('support', 1911), ('go', 1890), ('many', 1851), ('trump', 1809), ('say', 1794), ('pay', 1788), ('actually', 1785), ('lot', 1674), ('someone', 1659), ('better', 1626), ('take', 1599), ('new', 1554), ('sure', 1527), ('every', 1524), ('first', 1524), ('campaign', 1470), ('work', 1464), ('healthcare', 1458), ('biden', 1446), ('said', 1422), ('got', 1410), ('government', 1407)]
****************************************
r/elizabethwarren most common words: 
[('warren', 13893), ('would', 12487), ('p

In [ ]:
For Text Mining

In [10]:
#Bernie vs Trump

test_set_size = 5000

bernieset = [(desc_features(w), 'bernie') for w in cand_dict['sandersforpresident'][4] 
            if w]
trumpset = [(desc_features(w), 'trump') for w in cand_dict['the_donald'][4] if w]

bernie_trump_set = bernieset+trumpset

random.shuffle(bernie_trump_set)

train_set, test_set = bernie_trump_set[test_set_size:], bernie_trump_set[:test_set_size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [11]:
print(nltk.classify.accuracy(classifier, test_set))

0.6894


In [12]:
classifier.show_most_informative_features(20)

Most Informative Features
                  bernie = True           bernie : trump  =    171.7 : 1.0
                    pete = True           bernie : trump  =     99.8 : 1.0
                 ukraine = True            trump : bernie =     97.1 : 1.0
                medicare = True           bernie : trump  =     96.9 : 1.0
                 doctors = True           bernie : trump  =     47.1 : 1.0
                     jim = True            trump : bernie =     46.1 : 1.0
                 sanders = True           bernie : trump  =     46.0 : 1.0
                    iowa = True           bernie : trump  =     42.4 : 1.0
                  jordan = True            trump : bernie =     41.6 : 1.0
                 witness = True            trump : bernie =     40.9 : 1.0
                  caucus = True           bernie : trump  =     38.2 : 1.0
              healthcare = True           bernie : trump  =     35.6 : 1.0
                   islam = True            trump : bernie =     33.4 : 1.0

In [232]:
# Progressives vs Moderates

test_set_size = 5000

progset1 = [(desc_features(w), 'progressive') for w in cand_dict['sandersforpresident'][4] 
            if w]

progset2 = [(desc_features(w), 'progressive') for w in cand_dict['elizabethwarren'][4] 
            if w]

modset1 = [(desc_features(w), 'moderate') for w in cand_dict['pete_buttigieg'][4] if w]

modset2 = [(desc_features(w), 'moderate') for w in cand_dict['joebiden'][4] if w]

demset = progset1+progset2+modset1+modset2

random.shuffle(demset)

train_set, test_set = demset[test_set_size:], demset[:test_set_size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [233]:
print(nltk.classify.accuracy(classifier, test_set))

0.6868


In [236]:
classifier.show_most_informative_features(20)

Most Informative Features
                      lj = True           modera : progre =     58.9 : 1.0
                      aa = True           modera : progre =     27.7 : 1.0
                cinnamon = True           modera : progre =     22.0 : 1.0
               phonebank = True           progre : modera =     19.1 : 1.0
                  slaves = True           progre : modera =     19.1 : 1.0
                     vat = True           progre : modera =     18.8 : 1.0
             submissions = True           progre : modera =     18.6 : 1.0
                  busing = True           modera : progre =     18.0 : 1.0
                 mayoral = True           modera : progre =     16.6 : 1.0
                      sb = True           modera : progre =     15.4 : 1.0
                bolivian = True           progre : modera =     15.2 : 1.0
                 bullock = True           modera : progre =     14.9 : 1.0
             celebration = True           modera : progre =     14.8 : 1.0

In [14]:
# Democrats vs Trump

test_set_size = 5000

demset1 = [(desc_features(w), 'democrat') for w in cand_dict['sandersforpresident'][4] if w]

demset2 = [(desc_features(w), 'democrat') for w in cand_dict['elizabethwarren'][4] if w]

demset3 = [(desc_features(w), 'democrat') for w in cand_dict['pete_buttigieg'][4] if w]

demset4 = [(desc_features(w), 'democrat') for w in cand_dict['joebiden'][4] if w]

trumpset = [(desc_features(w), 'trump') for w in cand_dict['the_donald'][4] if w]

totalset = demset1+demset2+demset3+demset4+trumpset


random.shuffle(totalset)

train_set, test_set = totalset[test_set_size:], totalset[:test_set_size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [15]:
print(nltk.classify.accuracy(classifier, test_set))

0.8482


In [16]:
classifier.show_most_informative_features(20)

Most Informative Features
                    pete = True           democr : trump  =    751.4 : 1.0
                   islam = True            trump : democr =    155.5 : 1.0
                  jordan = True            trump : democr =    133.9 : 1.0
                 hearsay = True            trump : democr =    131.9 : 1.0
                  bernie = True           democr : trump  =    120.4 : 1.0
                  drudge = True            trump : democr =    118.5 : 1.0
                  taylor = True            trump : democr =    103.1 : 1.0
                     tds = True            trump : democr =     78.5 : 1.0
                    iowa = True           democr : trump  =     67.4 : 1.0
             bureaucrats = True            trump : democr =     60.0 : 1.0
                medicare = True           democr : trump  =     56.8 : 1.0
               witnesses = True            trump : democr =     51.8 : 1.0
                   mints = True            trump : democr =     50.8 : 1.0

In [244]:
# Bernie vs Warren

test_set_size = 5000

bernieset = [(desc_features(w), 'bernie') for w in cand_dict['sandersforpresident'][4] if w]

warrenset = [(desc_features(w), 'warren') for w in cand_dict['elizabethwarren'][4] if w]

totalset = bernieset+warrenset


random.shuffle(totalset)

train_set, test_set = totalset[test_set_size:], totalset[:test_set_size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [245]:
print(nltk.classify.accuracy(classifier, test_set))

0.6002


In [247]:
classifier.show_most_informative_features(20)

Most Informative Features
                 slavery = True           bernie : warren =     30.0 : 1.0
                    rand = True           bernie : warren =     26.1 : 1.0
                behavior = True           warren : bernie =     24.1 : 1.0
                    bite = True           warren : bernie =     23.6 : 1.0
                    lula = True           bernie : warren =     22.6 : 1.0
                     evo = True           bernie : warren =     21.7 : 1.0
                     cia = True           bernie : warren =     19.3 : 1.0
                   slave = True           bernie : warren =     18.5 : 1.0
                  empire = True           bernie : warren =     18.5 : 1.0
               phonebank = True           bernie : warren =     17.5 : 1.0
                 spanish = True           bernie : warren =     16.7 : 1.0
                 handled = True           warren : bernie =     16.0 : 1.0
                  slaves = True           bernie : warren =     13.5 : 1.0

In [248]:
# Remaining: 
# Warren vs Trump

test_set_size = 5000

warrenset = [(desc_features(w), 'warren') for w in cand_dict['elizabethwarren'][4] if w]

trumpset = [(desc_features(w), 'trump') for w in cand_dict['the_donald'][4] if w]

totalset = warrenset+trumpset


random.shuffle(totalset)

train_set, test_set = totalset[test_set_size:], totalset[:test_set_size]
classifier = nltk.NaiveBayesClassifier.train(train_set)



In [249]:
print(nltk.classify.accuracy(classifier, test_set))

0.6746


In [250]:
classifier.show_most_informative_features(20)

Most Informative Features
                    pete = True           warren : trump  =    255.9 : 1.0
                  bernie = True           warren : trump  =    129.1 : 1.0
                medicare = True           warren : trump  =    124.9 : 1.0
                     liz = True           warren : trump  =    107.8 : 1.0
                  jordan = True            trump : warren =     86.1 : 1.0
                  warren = True           warren : trump  =     86.0 : 1.0
                   polls = True           warren : trump  =     79.2 : 1.0
                 sanders = True           warren : trump  =     70.6 : 1.0
                   plans = True           warren : trump  =     59.9 : 1.0
              candidates = True           warren : trump  =     55.0 : 1.0
                   costs = True           warren : trump  =     53.3 : 1.0
                 endorse = True           warren : trump  =     48.0 : 1.0
                 doctors = True           warren : trump  =     47.0 : 1.0